In [1]:
import os
from datetime import date

import pandas as pd
from sqlalchemy import text
from sqlalchemy.engine.cursor import LegacyCursorResult

from app.db.config import db_engine, Session
from app.parser import Parser
from app.scraper import Scraper

In [2]:
with db_engine.connect() as conn:
    result: LegacyCursorResult = conn.execute(text('''
        SELECT * FROM city
    '''))

    for row in result:
        destination = row
        on_date = date(year=2021, month=12, day=2)
        break

In [3]:
destination, on_date

(('Будапешт', 'BUD'), datetime.date(2021, 12, 2))

In [4]:
scraper = Scraper(
    destination=destination,
    on_date=on_date
)
scraper.run()
HTML: str = scraper.HTML

In [5]:
df: pd.DataFrame = Parser.get_dataframe(
    destination=destination,
    on_date=on_date,
    HTML=HTML
)
    
df

,destination,parsing_date,departure_date,days_until,airlines,departure_time,arrival_time,duration_m,departure_airport,arrival_airport,min_price,stops_count
0,Будапешт,2021-11-23,2021-12-02,9,Wizz Air,11:05,11:45,160,VKO,BUD,12215,0
1,Будапешт,2021-11-23,2021-12-02,9,Aeroflot,10:45,11:35,170,SVO,BUD,17160,0
2,Будапешт,2021-11-23,2021-12-02,9,Finnair,13:35,18:30,415,SVO,BUD,11477,1
3,Будапешт,2021-11-23,2021-12-02,9,LOT,14:20,17:25,305,SVO,BUD,17810,1
4,Будапешт,2021-11-23,2021-12-02,9,KLM,17:55,23:05,430,SVO,BUD,15445,1
5,Будапешт,2021-11-23,2021-12-02,9,Finnair,21:00,10:50,950,SVO,BUD,8322,1
6,Будапешт,2021-11-23,2021-12-02,9,KLM,05:30,12:05,515,SVO,BUD,15445,1
7,Будапешт,2021-11-23,2021-12-02,9,Finnair,10:35,18:30,595,SVO,BUD,16968,1
8,Будапешт,2021-11-23,2021-12-02,9,LOT,21:15,11:25,970,SVO,BUD,11511,1
9,Будапешт,2021-11-23,2021-12-02,9,Turkish Airlines,13:00,19:25,505,VKO,BUD,23287,1


In [6]:
with Session() as session:
    df.to_sql(
        name='flight',
        con=session.get_bind(),
        if_exists='append',
        index=False
    )